# Quantum Hackathon

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator as RuntimeEstimator

# Replace with your actual API key
api_key = "FZ75ClkijC8JeJrgKLoGDji9jIOwLwUEO6w8A-nzI2H0"

service = QiskitRuntimeService(
    channel="ibm_cloud",
    token=api_key,
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/7fc8a37b01dc43198b31f8454a7806a3:a8bd70ec-2f03-454d-8d33-beeb7d85e31b::"
)

# List available backends to confirm (your dashboard shows ibm_fez, ibm_torino, ibm_marrakesh)
print(service.backends())

# Select a backend (e.g., the 156-qubit ibm_fez; use least_busy for minimal queue)
backend = service.backend("ibm_fez")  # Or: service.least_busy(operational=True, simulator=False, min_num_qubits=10)

qiskit_runtime_service._discover_account:WARNING:2025-11-08 18:16:19,832: Loading account with the given token. A saved account will not be used.


[<IBMBackend('ibm_fez')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_marrakesh')>]


In [3]:
"""
Quantum Protein Folding Analysis with VQE
==========================================
Analyzes SARS-CoV-2 protein regions using Variational Quantum Eigensolver
on a 2D HP lattice model. Adapted for IBM Quantum hardware.

Requirements:
pip install qiskit qiskit-aer qiskit-algorithms qiskit-ibm-runtime matplotlib numpy scipy
"""

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler import PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes.scheduling import ALAPScheduleAnalysis, PadDynamicalDecoupling
from qiskit.circuit.library import XGate
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as RuntimeEstimator
import time

In [4]:
# ============================================================================
# PROTEIN DATA
# ============================================================================

HIGH_CONFIDENCE_TARGET = {
    'name': 'SARS-CoV-2 RBD Core (Structured)',
    'sequence': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF',
    'length': 223,
    'region': 'Full RBD domain',
}

LOW_CONFIDENCE_TARGET = {
    'name': 'SARS-CoV-2 Omicron BA.5 Flexible Loop',
    'sequence': 'YQPYRVVVLS',
    'length': 10,
    'region': 'Extended loop in RBD (residues 493-502)',
}

In [5]:
# ============================================================================
# HP MODEL: Convert amino acids to Hydrophobic (1) or Polar (0)
# ============================================================================

def sequence_to_hp(sequence):
    """
    Convert amino acid sequence to HP (Hydrophobic-Polar) model.
    
    Hydrophobic (H=1): A, V, I, L, M, F, Y, W
    Polar (P=0): R, N, D, C, Q, E, G, H, K, P, S, T
    """
    hydrophobic = set('AVILMFYW')
    hp_sequence = []
    
    for aa in sequence:
        if aa in hydrophobic:
            hp_sequence.append(1)  # H
        else:
            hp_sequence.append(0)  # P
    
    return np.array(hp_sequence)

def print_hp_sequence(name, sequence, hp_seq):
    """Visualize HP mapping"""
    print(f"\n{'='*80}")
    print(f"🧬 {name}")
    print(f"{'='*80}")
    print(f"Length: {len(sequence)} residues")
    print(f"\nHP Pattern:")
    
    # Print first 50 for visualization
    display_length = min(50, len(sequence))
    print(f"Sequence: {sequence[:display_length]}...")
    print(f"HP Code:  {''.join(['H' if h else 'P' for h in hp_seq[:display_length]])}...")
    
    h_count = np.sum(hp_seq)
    p_count = len(hp_seq) - h_count
    print(f"\n📊 Composition:")
    print(f"   Hydrophobic (H): {h_count} ({100*h_count/len(hp_seq):.1f}%)")
    print(f"   Polar (P):       {p_count} ({100*p_count/len(hp_seq):.1f}%)")
    
    return hp_seq

In [6]:
# ============================================================================
# 2D LATTICE HAMILTONIAN
# ============================================================================

def create_lattice_hamiltonian(hp_sequence, contact_energy=-1.0):
    """
    Create a simplified Hamiltonian for HP lattice model.
    
    Energy function:
    E = Σ ε_ij for all non-adjacent contacts
    where ε_ij = -1 if both H-H contact, 0 otherwise
    
    For simplicity, we'll create a toy model with n qubits representing
    n positions, and measure interactions between H residues.
    """
    n = len(hp_sequence)
    
    # For a tractable VQE problem, we'll encode interactions as Pauli strings
    # Each qubit represents whether a hydrophobic residue is "active" in forming contacts
    
    pauli_list = []
    
    # H-H contact interactions (simplified: nearest neighbors + diagonal)
    for i in range(n - 1):
        if hp_sequence[i] == 1 and hp_sequence[i+1] == 1:
            # H-H contact: favorable interaction
            # Z_i Z_{i+1} measures correlation
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_str[i+1] = 'Z'
            pauli_list.append((''.join(pauli_str), contact_energy))
    
    # Add some longer-range interactions for realism
    for i in range(n - 2):
        if hp_sequence[i] == 1 and hp_sequence[i+2] == 1:
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_str[i+2] = 'Z'
            pauli_list.append((''.join(pauli_str), contact_energy * 0.5))
    
    # Entropic penalty (disfavors too many active contacts - represents entropy loss)
    for i in range(n):
        if hp_sequence[i] == 1:
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_list.append((''.join(pauli_str), 0.3))
    
    if len(pauli_list) == 0:
        # No interactions (all polar) - add identity
        pauli_list.append(('I' * n, 0.0))
    
    hamiltonian = SparsePauliOp.from_list(pauli_list)
    
    return hamiltonian

In [7]:
# ============================================================================
# VARIATIONAL ANSATZ
# ============================================================================

def create_ansatz(num_qubits, layers=2):
    """
    Create a hardware-efficient ansatz for VQE.
    Uses RY rotations and CNOT entanglement.
    """
    qc = QuantumCircuit(num_qubits)
    params = []
    
    # Initial layer of rotations
    for i in range(num_qubits):
        param = Parameter(f'θ_init_{i}')
        qc.ry(param, i)
        params.append(param)
    
    # Entangling layers
    for layer in range(layers):
        # Entanglement
        for i in range(num_qubits - 1):
            qc.cx(i, i + 1)
        
        # Rotations
        for i in range(num_qubits):
            param = Parameter(f'θ_L{layer}_{i}')
            qc.ry(param, i)
            params.append(param)
    
    return qc, params

In [8]:
# ============================================================================
# VQE OPTIMIZATION
# ============================================================================

def run_vqe_analysis(name, hp_sequence, max_qubits=8, api_key="YOUR_IBM_CLOUD_API_KEY"):
    """
    Run VQE to find ground state energy of HP lattice model on IBM Quantum hardware.
    """
    print(f"\n{'='*80}")
    print(f"⚛️  QUANTUM VQE ANALYSIS: {name}")
    print(f"{'='*80}")
    
    # Truncate if too long
    if len(hp_sequence) > max_qubits:
        print(f"⚠️  Sequence too long ({len(hp_sequence)} residues)")
        print(f"   Using first {max_qubits} residues for quantum analysis")
        hp_sequence = hp_sequence[:max_qubits]
    
    num_qubits = len(hp_sequence)
    print(f"\n🔬 VQE Setup:")
    print(f"   Qubits: {num_qubits}")
    print(f"   HP Sequence: {''.join(['H' if h else 'P' for h in hp_sequence])}")
    
    # Create Hamiltonian
    print(f"\n📐 Building Hamiltonian...")
    hamiltonian = create_lattice_hamiltonian(hp_sequence)
    print(f"   Pauli terms: {len(hamiltonian)}")
    print(f"   Sample terms: {list(hamiltonian.to_list())[:3]}")
    
    # Create ansatz
    print(f"\n🎛️  Creating variational circuit...")
    ansatz, params = create_ansatz(num_qubits, layers=2)
    num_params = len(params)
    print(f"   Parameters: {num_params}")
    print(f"   Circuit depth: {ansatz.depth()}")
    
    # Setup Qiskit Runtime Service
    service = QiskitRuntimeService(
        channel="ibm_cloud",
        token=api_key,
        instance="crn:v1:bluemix:public:quantum-computing:us-east:a/7fc8a37b01dc43198b31f8454a7806a3:a8bd70ec-2f03-454d-8d33-beeb7d85e31b::"
    )
    
    # Select optimal backend
    backends = service.backends(simulator=False, operational=True, min_num_qubits=10)
    best_backend = None
    lowest_error = float('inf')
    for b in backends:
        if b.name in ['ibm_marrakesh', 'ibm_fez', 'ibm_torino']:
            props = b.properties()
            cx_errors = [gate.error for gate in props.gates if gate.gate == 'cx']
            avg_cx_error = sum(cx_errors) / len(cx_errors) if cx_errors else float('inf')
            if avg_cx_error < lowest_error:
                lowest_error = avg_cx_error
                best_backend = b

    if not best_backend:
        best_backend = service.least_busy(operational=True, simulator=False, min_num_qubits=10)

    print(f"Selected backend: {best_backend.name} (Avg CNOT error: {lowest_error:.2e})")
    
    # Transpile ansatz for hardware
    target = best_backend.target
    pm = generate_preset_pass_manager(optimization_level=3, backend=best_backend)
    
    # Add dynamical decoupling
    dd_pm = PassManager([
        ALAPScheduleAnalysis(durations=target.durations()),
        PadDynamicalDecoupling(
            durations=target.durations(),
            dd_sequence=[XGate(), XGate()],
            pulse_alignment=target.pulse_alignment
        )
    ])
    pm.append(dd_pm)
    
    ansatz_transpiled = pm.run(ansatz)
    
    # Apply layout to Hamiltonian
    hamiltonian_transpiled = hamiltonian.apply_layout(ansatz_transpiled.layout)
    
    # Set up Runtime Estimator V2
    print(f"\n⚡ Running VQE optimization on hardware...")
    estimator = RuntimeEstimator(mode=best_backend)
    estimator.options.default_shots = 1024
    estimator.options.resilience.measure_mitigation = True  # TREX
    # estimator.options.resilience.zne_mitigation = True  # ZNE (optional, costs more)
    # estimator.options.resilience.zne.extrapolator = 'linear'
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = 'XX'
    
    optimizer = COBYLA(maxiter=50)  # Reduced for time constraints
    
    # Initial point
    np.random.seed(42)
    initial_point = np.random.uniform(-0.1, 0.1, num_params)
    
    vqe = VQE(estimator, ansatz_transpiled, optimizer, initial_point=initial_point)
    
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(hamiltonian_transpiled)
    end_time = time.time()
    
    # Extract results
    energy = result.optimal_value
    optimal_params = result.optimal_point
    
    print(f"\n✅ VQE Complete!")
    print(f"   Time: {end_time - start_time:.2f} seconds")
    print(f"   Ground state energy: {energy:.4f}")
    print(f"   Optimizer iterations: {result.cost_function_evals}")
    
    # Analyze stability
    stability_score = -energy  # Lower energy = more stable = higher score
    
    print(f"\n📊 Structural Analysis:")
    print(f"   Stability score: {stability_score:.4f}")
    
    if stability_score > 2.0:
        stability = "HIGH (well-folded, stable)"
    elif stability_score > 1.0:
        stability = "MEDIUM (partially stable)"
    else:
        stability = "LOW (disordered, flexible)"
    
    print(f"   Interpretation: {stability}")
    
    return {
        'name': name,
        'hp_sequence': hp_sequence,
        'num_qubits': num_qubits,
        'energy': energy,
        'stability_score': stability_score,
        'stability': stability,
        'optimal_params': optimal_params,
        'time': end_time - start_time,
        'iterations': result.cost_function_evals
    }

In [9]:
# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_comparison(results_high, results_low):
    """
    Compare quantum VQE results for high vs low confidence regions.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('🧬 Quantum VQE Analysis: SARS-CoV-2 Protein Regions', 
                 fontsize=16, fontweight='bold')
    
    # Stability comparison
    ax1 = axes[0, 0]
    names = [results_high['name'].split('(')[0].strip(), 
             results_low['name'].split('(')[0].strip()]
    stabilities = [results_high['stability_score'], results_low['stability_score']]
    colors = ['green' if s > 1.5 else 'orange' if s > 0.8 else 'red' for s in stabilities]
    
    bars = ax1.bar(names, stabilities, color=colors, alpha=0.7, edgecolor='black')
    ax1.set_ylabel('Stability Score (Higher = More Stable)', fontweight='bold')
    ax1.set_title('Ground State Stability Comparison')
    ax1.axhline(y=1.5, color='green', linestyle='--', alpha=0.3, label='High stability threshold')
    ax1.axhline(y=0.8, color='orange', linestyle='--', alpha=0.3, label='Medium stability threshold')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, stabilities):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.2f}',
                ha='center', va='bottom', fontweight='bold')
    
    # Energy comparison
    ax2 = axes[0, 1]
    energies = [results_high['energy'], results_low['energy']]
    bars = ax2.bar(names, energies, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax2.set_ylabel('Ground State Energy (Lower = More Stable)', fontweight='bold')
    ax2.set_title('VQE Ground State Energy')
    ax2.grid(axis='y', alpha=0.3)
    
    for bar, val in zip(bars, energies):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}',
                ha='center', va='bottom' if val < 0 else 'top', fontweight='bold')
    
    # HP sequence patterns
    ax3 = axes[1, 0]
    hp_high = results_high['hp_sequence']
    hp_low = results_low['hp_sequence']
    
    ax3.imshow([hp_high], cmap='RdYlGn', aspect='auto', interpolation='nearest')
    ax3.set_yticks([0])
    ax3.set_yticklabels([results_high['name'].split('(')[0].strip()])
    ax3.set_xlabel('Residue Position')
    ax3.set_title('HP Pattern: High Confidence Region')
    ax3.set_xticks(range(len(hp_high)))
    ax3.set_xticklabels(['H' if h else 'P' for h in hp_high], fontsize=8)
    
    ax4 = axes[1, 1]
    ax4.imshow([hp_low], cmap='RdYlGn', aspect='auto', interpolation='nearest')
    ax4.set_yticks([0])
    ax4.set_yticklabels([results_low['name'].split('(')[0].strip()])
    ax4.set_xlabel('Residue Position')
    ax4.set_title('HP Pattern: Low Confidence Region')
    ax4.set_xticks(range(len(hp_low)))
    ax4.set_xticklabels(['H' if h else 'P' for h in hp_low], fontsize=10)
    
    plt.tight_layout()
    plt.savefig('vqe_protein_analysis.png', dpi=300, bbox_inches='tight')
    print("\n💾 Visualization saved as 'vqe_protein_analysis.png'")
    plt.show()

In [10]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main(api_key="FZ75ClkijC8JeJrgKLoGDji9jIOwLwUEO6w8A-nzI2H0"):
    print("\n" + "="*80)
    print("⚛️  QUANTUM PROTEIN FOLDING WITH VQE")
    print("="*80)
    print("\n🔬 Method: Variational Quantum Eigensolver (VQE)")
    print("📐 Model: HP Lattice (Hydrophobic-Polar)")
    print("💻 Backend: IBM Quantum Hardware")
    print("⏱️  Target runtime: ~8 minutes")
    
    # Convert sequences to HP model
    hp_high = print_hp_sequence(
        HIGH_CONFIDENCE_TARGET['name'],
        HIGH_CONFIDENCE_TARGET['sequence'],
        sequence_to_hp(HIGH_CONFIDENCE_TARGET['sequence'])
    )
    
    hp_low = print_hp_sequence(
        LOW_CONFIDENCE_TARGET['name'],
        LOW_CONFIDENCE_TARGET['sequence'],
        sequence_to_hp(LOW_CONFIDENCE_TARGET['sequence'])
    )
    
    print("\n" + "="*80)
    print("🚀 Starting VQE Quantum Simulations...")
    print("="*80)
    
    # Run VQE for high confidence region (fragment)
    results_high = run_vqe_analysis(
        HIGH_CONFIDENCE_TARGET['name'],
        hp_high,
        max_qubits=8,  # Use 8-residue fragment
        api_key=api_key
    )
    
    # Run VQE for low confidence region
    results_low = run_vqe_analysis(
        LOW_CONFIDENCE_TARGET['name'],
        hp_low,
        max_qubits=10,  # Can handle full 10 residues
        api_key=api_key
    )
    
    # Summary
    print("\n" + "="*80)
    print("📊 QUANTUM ANALYSIS SUMMARY")
    print("="*80)
    
    print(f"\n🧬 {results_high['name']}")
    print(f"   Ground State Energy: {results_high['energy']:.4f}")
    print(f"   Stability Score: {results_high['stability_score']:.4f}")
    print(f"   Classification: {results_high['stability']}")
    print(f"   VQE Runtime: {results_high['time']:.2f}s")
    
    print(f"\n🧬 {results_low['name']}")
    print(f"   Ground State Energy: {results_low['energy']:.4f}")
    print(f"   Stability Score: {results_low['stability_score']:.4f}")
    print(f"   Classification: {results_low['stability']}")
    print(f"   VQE Runtime: {results_low['time']:.2f}s")
    
    print("\n" + "="*80)
    print("🎯 BIOLOGICAL INTERPRETATION")
    print("="*80)
    
    if results_high['stability_score'] > results_low['stability_score']:
        print("\n✅ EXPECTED RESULT CONFIRMED!")
        print(f"   The structured RBD core shows HIGHER stability ({results_high['stability_score']:.2f})")
        print(f"   The flexible loop shows LOWER stability ({results_low['stability_score']:.2f})")
        print("\n   This quantum simulation confirms that:")
        print("   • The RBD core has favorable hydrophobic interactions")
        print("   • The flexible loop is less energetically stable")
        print("   • Structural predictions align with quantum energy landscape")
    else:
        print("\n⚠️  UNEXPECTED RESULT")
        print("   Further analysis needed - could indicate:")
        print("   • Limitations of simplified HP model")
        print("   • Need for longer VQE optimization")
        print("   • Importance of longer-range interactions")
    
    print("\n" + "="*80)
    print("📈 Generating visualizations...")
    plot_comparison(results_high, results_low)
    
    print("\n✅ Analysis complete!")
    print("="*80)

In [11]:
main(api_key="FZ75ClkijC8JeJrgKLoGDji9jIOwLwUEO6w8A-nzI2H0")

qiskit_runtime_service._discover_account:WARNING:2025-11-08 18:23:10,574: Loading account with the given token. A saved account will not be used.



⚛️  QUANTUM PROTEIN FOLDING WITH VQE

🔬 Method: Variational Quantum Eigensolver (VQE)
📐 Model: HP Lattice (Hydrophobic-Polar)
💻 Backend: IBM Quantum Hardware
⏱️  Target runtime: ~8 minutes

🧬 SARS-CoV-2 RBD Core (Structured)
Length: 223 residues

HP Pattern:
Sequence: RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVL...
HP Code:  PHPPPPPHHPHPPHPPHPPHPPHHPHPPHHPHHHHPPPPHPPPHHPHPHH...

📊 Composition:
   Hydrophobic (H): 88 (39.5%)
   Polar (P):       135 (60.5%)

🧬 SARS-CoV-2 Omicron BA.5 Flexible Loop
Length: 10 residues

HP Pattern:
Sequence: YQPYRVVVLS...
HP Code:  HPPHPHHHHP...

📊 Composition:
   Hydrophobic (H): 6 (60.0%)
   Polar (P):       4 (40.0%)

🚀 Starting VQE Quantum Simulations...

⚛️  QUANTUM VQE ANALYSIS: SARS-CoV-2 RBD Core (Structured)
⚠️  Sequence too long (223 residues)
   Using first 8 residues for quantum analysis

🔬 VQE Setup:
   Qubits: 8
   HP Sequence: PHPPPPPH

📐 Building Hamiltonian...
   Pauli terms: 2
   Sample terms: [('IZIIIIII', (0.3+0j)), ('IIIIIIIZ', 

NotImplementedError: 

### New trial

In [ ]:
"""
Quantum Protein Folding Analysis with VQE
==========================================
Analyzes SARS-CoV-2 protein regions using Variational Quantum Eigensolver
on a 2D HP lattice model. Adapted for IBM Quantum hardware.

Requirements:
pip install qiskit qiskit-aer qiskit-algorithms qiskit-ibm-runtime matplotlib numpy scipy
"""

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as RuntimeEstimator
import time

# ============================================================================
# PROTEIN DATA
# ============================================================================

HIGH_CONFIDENCE_TARGET = {
    'name': 'SARS-CoV-2 RBD Core (Structured)',
    'sequence': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF',
    'length': 223,
    'region': 'Full RBD domain',
}

LOW_CONFIDENCE_TARGET = {
    'name': 'SARS-CoV-2 Omicron BA.5 Flexible Loop',
    'sequence': 'YQPYRVVVLS',
    'length': 10,
    'region': 'Extended loop in RBD (residues 493-502)',
}

# ============================================================================
# HP MODEL: Convert amino acids to Hydrophobic (1) or Polar (0)
# ============================================================================

def sequence_to_hp(sequence):
    """
    Convert amino acid sequence to HP (Hydrophobic-Polar) model.
    
    Hydrophobic (H=1): A, V, I, L, M, F, Y, W
    Polar (P=0): R, N, D, C, Q, E, G, H, K, P, S, T
    """
    hydrophobic = set('AVILMFYW')
    hp_sequence = []
    
    for aa in sequence:
        if aa in hydrophobic:
            hp_sequence.append(1)  # H
        else:
            hp_sequence.append(0)  # P
    
    return np.array(hp_sequence)

def print_hp_sequence(name, sequence, hp_seq):
    """Visualize HP mapping"""
    print(f"\n{'='*80}")
    print(f"🧬 {name}")
    print(f"{'='*80}")
    print(f"Length: {len(sequence)} residues")
    print(f"\nHP Pattern:")
    
    # Print first 50 for visualization
    display_length = min(50, len(sequence))
    print(f"Sequence: {sequence[:display_length]}...")
    print(f"HP Code:  {''.join(['H' if h else 'P' for h in hp_seq[:display_length]])}...")
    
    h_count = np.sum(hp_seq)
    p_count = len(hp_seq) - h_count
    print(f"\n📊 Composition:")
    print(f"   Hydrophobic (H): {h_count} ({100*h_count/len(hp_seq):.1f}%)")
    print(f"   Polar (P):       {p_count} ({100*p_count/len(hp_seq):.1f}%)")
    
    return hp_seq

# ============================================================================
# 2D LATTICE HAMILTONIAN
# ============================================================================

def create_lattice_hamiltonian(hp_sequence, contact_energy=-1.0):
    """
    Create a simplified Hamiltonian for HP lattice model.
    
    Energy function:
    E = Σ ε_ij for all non-adjacent contacts
    where ε_ij = -1 if both H-H contact, 0 otherwise
    
    For simplicity, we'll create a toy model with n qubits representing
    n positions, and measure interactions between H residues.
    """
    n = len(hp_sequence)
    
    # For a tractable VQE problem, we'll encode interactions as Pauli strings
    # Each qubit represents whether a hydrophobic residue is "active" in forming contacts
    
    pauli_list = []
    
    # H-H contact interactions (simplified: nearest neighbors + diagonal)
    for i in range(n - 1):
        if hp_sequence[i] == 1 and hp_sequence[i+1] == 1:
            # H-H contact: favorable interaction
            # Z_i Z_{i+1} measures correlation
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_str[i+1] = 'Z'
            pauli_list.append((''.join(pauli_str), contact_energy))
    
    # Add some longer-range interactions for realism
    for i in range(n - 2):
        if hp_sequence[i] == 1 and hp_sequence[i+2] == 1:
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_str[i+2] = 'Z'
            pauli_list.append((''.join(pauli_str), contact_energy * 0.5))
    
    # Entropic penalty (disfavors too many active contacts - represents entropy loss)
    for i in range(n):
        if hp_sequence[i] == 1:
            pauli_str = ['I'] * n
            pauli_str[i] = 'Z'
            pauli_list.append((''.join(pauli_str), 0.3))
    
    if len(pauli_list) == 0:
        # No interactions (all polar) - add identity
        pauli_list.append(('I' * n, 0.0))
    
    hamiltonian = SparsePauliOp.from_list(pauli_list)
    
    return hamiltonian

# ============================================================================
# VARIATIONAL ANSATZ
# ============================================================================

def create_ansatz(num_qubits, layers=2):
    """
    Create a hardware-efficient ansatz for VQE.
    Uses RY rotations and CNOT entanglement.
    """
    qc = QuantumCircuit(num_qubits)
    params = []
    
    # Initial layer of rotations
    for i in range(num_qubits):
        param = Parameter(f'θ_init_{i}')
        qc.ry(param, i)
        params.append(param)
    
    # Entangling layers
    for layer in range(layers):
        # Entanglement
        for i in range(num_qubits - 1):
            qc.cx(i, i + 1)
        
        # Rotations
        for i in range(num_qubits):
            param = Parameter(f'θ_L{layer}_{i}')
            qc.ry(param, i)
            params.append(param)
    
    return qc, params

# ============================================================================
# VQE OPTIMIZATION
# ============================================================================

def run_vqe_analysis(name, hp_sequence, max_qubits=8, api_key="YOUR_IBM_CLOUD_API_KEY"):
    """
    Run VQE to find ground state energy of HP lattice model on IBM Quantum hardware.
    """
    print(f"\n{'='*80}")
    print(f"⚛️  QUANTUM VQE ANALYSIS: {name}")
    print(f"{'='*80}")
    
    # Truncate if too long
    if len(hp_sequence) > max_qubits:
        print(f"⚠️  Sequence too long ({len(hp_sequence)} residues)")
        print(f"   Using first {max_qubits} residues for quantum analysis")
        hp_sequence = hp_sequence[:max_qubits]
    
    num_qubits = len(hp_sequence)
    print(f"\n🔬 VQE Setup:")
    print(f"   Qubits: {num_qubits}")
    print(f"   HP Sequence: {''.join(['H' if h else 'P' for h in hp_sequence])}")
    
    # Create Hamiltonian
    print(f"\n📐 Building Hamiltonian...")
    hamiltonian = create_lattice_hamiltonian(hp_sequence)
    print(f"   Pauli terms: {len(hamiltonian)}")
    print(f"   Sample terms: {list(hamiltonian.to_list())[:3]}")
    
    # Create ansatz
    print(f"\n🎛️  Creating variational circuit...")
    ansatz, params = create_ansatz(num_qubits, layers=2)
    num_params = len(params)
    print(f"   Parameters: {num_params}")
    print(f"   Circuit depth: {ansatz.depth()}")
    
    # Setup Qiskit Runtime Service
    service = QiskitRuntimeService(
        channel="ibm_cloud",
        token=api_key,
        instance="crn:v1:bluemix:public:quantum-computing:us-east:a/7fc8a37b01dc43198b31f8454a7806a3:a8bd70ec-2f03-454d-8d33-beeb7d85e31b::"
    )
    
    # Select optimal backend
    backends = service.backends(simulator=False, operational=True, min_num_qubits=10)
    best_backend = None
    lowest_error = float('inf')
    for b in backends:
        if b.name in ['ibm_marrakesh', 'ibm_fez', 'ibm_torino']:
            props = b.properties()
            cx_errors = [gate.error for gate in props.gates if gate.gate == 'cx']
            avg_cx_error = sum(cx_errors) / len(cx_errors) if cx_errors else float('inf')
            if avg_cx_error < lowest_error:
                lowest_error = avg_cx_error
                best_backend = b

    if not best_backend:
        best_backend = service.least_busy(operational=True, simulator=False, min_num_qubits=10)

    print(f"Selected backend: {best_backend.name} (Avg CNOT error: {lowest_error:.2e})")
    
    # Transpile ansatz for hardware (no custom DD append; handled by runtime options)
    pm = generate_preset_pass_manager(optimization_level=3, backend=best_backend)
    ansatz_transpiled = pm.run(ansatz)
    
    # Apply layout to Hamiltonian
    hamiltonian_transpiled = hamiltonian.apply_layout(ansatz_transpiled.layout)
    
    # Set up Runtime Estimator V2
    print(f"\n⚡ Running VQE optimization on hardware...")
    estimator = RuntimeEstimator(mode=best_backend)
    estimator.options.default_shots = 1024
    estimator.options.resilience.measure_mitigation = True  # TREX
    # estimator.options.resilience.zne_mitigation = True  # ZNE (optional, costs more)
    # estimator.options.resilience.zne.extrapolator = 'linear'
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = 'XX'
    
    optimizer = COBYLA(maxiter=50)  # Reduced for time constraints
    
    # Initial point
    np.random.seed(42)
    initial_point = np.random.uniform(-0.1, 0.1, num_params)
    
    vqe = VQE(estimator, ansatz_transpiled, optimizer, initial_point=initial_point)
    
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(hamiltonian_transpiled)
    end_time = time.time()
    
    # Extract results
    energy = result.optimal_value
    optimal_params = result.optimal_point
    
    print(f"\n✅ VQE Complete!")
    print(f"   Time: {end_time - start_time:.2f} seconds")
    print(f"   Ground state energy: {energy:.4f}")
    print(f"   Optimizer iterations: {result.cost_function_evals}")
    
    # Analyze stability
    stability_score = -energy  # Lower energy = more stable = higher score
    
    print(f"\n📊 Structural Analysis:")
    print(f"   Stability score: {stability_score:.4f}")
    
    if stability_score > 2.0:
        stability = "HIGH (well-folded, stable)"
    elif stability_score > 1.0:
        stability = "MEDIUM (partially stable)"
    else:
        stability = "LOW (disordered, flexible)"
    
    print(f"   Interpretation: {stability}")
    
    return {
        'name': name,
        'hp_sequence': hp_sequence,
        'num_qubits': num_qubits,
        'energy': energy,
        'stability_score': stability_score,
        'stability': stability,
        'optimal_params': optimal_params,
        'time': end_time - start_time,
        'iterations': result.cost_function_evals
    }

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_comparison(results_high, results_low):
    """
    Compare quantum VQE results for high vs low confidence regions.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('🧬 Quantum VQE Analysis: SARS-CoV-2 Protein Regions', 
                 fontsize=16, fontweight='bold')
    
    # Stability comparison
    ax1 = axes[0, 0]
    names = [results_high['name'].split('(')[0].strip(), 
             results_low['name'].split('(')[0].strip()]
    stabilities = [results_high['stability_score'], results_low['stability_score']]
    colors = ['green' if s > 1.5 else 'orange' if s > 0.8 else 'red' for s in stabilities]
    
    bars = ax1.bar(names, stabilities, color=colors, alpha=0.7, edgecolor='black')
    ax1.set_ylabel('Stability Score (Higher = More Stable)', fontweight='bold')
    ax1.set_title('Ground State Stability Comparison')
    ax1.axhline(y=1.5, color='green', linestyle='--', alpha=0.3, label='High stability threshold')
    ax1.axhline(y=0.8, color='orange', linestyle='--', alpha=0.3, label='Medium stability threshold')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, stabilities):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.2f}',
                ha='center', va='bottom', fontweight='bold')
    
    # Energy comparison
    ax2 = axes[0, 1]
    energies = [results_high['energy'], results_low['energy']]
    bars = ax2.bar(names, energies, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax2.set_ylabel('Ground State Energy (Lower = More Stable)', fontweight='bold')
    ax2.set_title('VQE Ground State Energy')
    ax2.grid(axis='y', alpha=0.3)
    
    for bar, val in zip(bars, energies):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}',
                ha='center', va='bottom' if val < 0 else 'top', fontweight='bold')
    
    # HP sequence patterns
    ax3 = axes[1, 0]
    hp_high = results_high['hp_sequence']
    hp_low = results_low['hp_sequence']
    
    ax3.imshow([hp_high], cmap='RdYlGn', aspect='auto', interpolation='nearest')
    ax3.set_yticks([0])
    ax3.set_yticklabels([results_high['name'].split('(')[0].strip()])
    ax3.set_xlabel('Residue Position')
    ax3.set_title('HP Pattern: High Confidence Region')
    ax3.set_xticks(range(len(hp_high)))
    ax3.set_xticklabels(['H' if h else 'P' for h in hp_high], fontsize=8)
    
    ax4 = axes[1, 1]
    ax4.imshow([hp_low], cmap='RdYlGn', aspect='auto', interpolation='nearest')
    ax4.set_yticks([0])
    ax4.set_yticklabels([results_low['name'].split('(')[0].strip()])
    ax4.set_xlabel('Residue Position')
    ax4.set_title('HP Pattern: Low Confidence Region')
    ax4.set_xticks(range(len(hp_low)))
    ax4.set_xticklabels(['H' if h else 'P' for h in hp_low], fontsize=10)
    
    plt.tight_layout()
    plt.savefig('vqe_protein_analysis.png', dpi=300, bbox_inches='tight')
    print("\n💾 Visualization saved as 'vqe_protein_analysis.png'")
    plt.show()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main(api_key="YOUR_IBM_CLOUD_API_KEY"):
    print("\n" + "="*80)
    print("⚛️  QUANTUM PROTEIN FOLDING WITH VQE")
    print("="*80)
    print("\n🔬 Method: Variational Quantum Eigensolver (VQE)")
    print("📐 Model: HP Lattice (Hydrophobic-Polar)")
    print("💻 Backend: IBM Quantum Hardware")
    print("⏱️  Target runtime: ~8 minutes")
    
    # Convert sequences to HP model
    hp_high = print_hp_sequence(
        HIGH_CONFIDENCE_TARGET['name'],
        HIGH_CONFIDENCE_TARGET['sequence'],
        sequence_to_hp(HIGH_CONFIDENCE_TARGET['sequence'])
    )
    
    hp_low = print_hp_sequence(
        LOW_CONFIDENCE_TARGET['name'],
        LOW_CONFIDENCE_TARGET['sequence'],
        sequence_to_hp(LOW_CONFIDENCE_TARGET['sequence'])
    )
    
    print("\n" + "="*80)
    print("🚀 Starting VQE Quantum Simulations...")
    print("="*80)
    
    # Run VQE for high confidence region (fragment)
    results_high = run_vqe_analysis(
        HIGH_CONFIDENCE_TARGET['name'],
        hp_high,
        max_qubits=8,  # Use 8-residue fragment
        api_key=api_key
    )
    
    # Run VQE for low confidence region
    results_low = run_vqe_analysis(
        LOW_CONFIDENCE_TARGET['name'],
        hp_low,
        max_qubits=10,  # Can handle full 10 residues
        api_key=api_key
    )
    
    # Summary
    print("\n" + "="*80)
    print("📊 QUANTUM ANALYSIS SUMMARY")
    print("="*80)
    
    print(f"\n🧬 {results_high['name']}")
    print(f"   Ground State Energy: {results_high['energy']:.4f}")
    print(f"   Stability Score: {results_high['stability_score']:.4f}")
    print(f"   Classification: {results_high['stability']}")
    print(f"   VQE Runtime: {results_high['time']:.2f}s")
    
    print(f"\n🧬 {results_low['name']}")
    print(f"   Ground State Energy: {results_low['energy']:.4f}")
    print(f"   Stability Score: {results_low['stability_score']:.4f}")
    print(f"   Classification: {results_low['stability']}")
    print(f"   VQE Runtime: {results_low['time']:.2f}s")
    
    print("\n" + "="*80)
    print("🎯 BIOLOGICAL INTERPRETATION")
    print("="*80)
    
    if results_high['stability_score'] > results_low['stability_score']:
        print("\n✅ EXPECTED RESULT CONFIRMED!")
        print(f"   The structured RBD core shows HIGHER stability ({results_high['stability_score']:.2f})")
        print(f"   The flexible loop shows LOWER stability ({results_low['stability_score']:.2f})")
        print("\n   This quantum simulation confirms that:")
        print("   • The RBD core has favorable hydrophobic interactions")
        print("   • The flexible loop is less energetically stable")
        print("   • Structural predictions align with quantum energy landscape")
    else:
        print("\n⚠️  UNEXPECTED RESULT")
        print("   Further analysis needed - could indicate:")
        print("   • Limitations of simplified HP model")
        print("   • Need for longer VQE optimization")
        print("   • Importance of longer-range interactions")
    
    print("\n" + "="*80)
    print("📈 Generating visualizations...")
    plot_comparison(results_high, results_low)
    
    print("\n✅ Analysis complete!")
    print("="*80)



In [ ]:
# To run in Jupyter, call main with your API key
main(api_key="FZ75ClkijC8JeJrgKLoGDji9jIOwLwUEO6w8A-nzI2H0")